# Export 2019 Recipients and create xlsx-Files

In [1]:
import pandas as pd
import mysql.connector

In [2]:
year = 2019

## Connect

In [3]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [4]:
df_recipient = pd.read_sql("""SELECT
  rec_id as id,
  rec_name as name,
  rec_address as address,
  rec_location as location,
  rec_type as type,
  trc_name as category,
  sum(tra_value) as value
 FROM transaction
  LEFT JOIN transaction_category ON tra_fk_transaction_category = transaction_category.trc_id
  LEFT JOIN recipient ON tra_fk_recipient = recipient.rec_id
  WHERE tra_year = %s
  GROUP BY tra_fk_recipient, tra_fk_transaction_category
  ORDER BY rec_name DESC""" % year, con=db_connection)

## Export

In [5]:
df_hcp = df_recipient[df_recipient.type == 'hcp']
df_hco = df_recipient[df_recipient.type == 'hco']

#Pivot HCP
df_pivot_hcp = pd.pivot_table(df_hcp, values='value', columns='category', index=['id', 'name', 'address', 'location'])
df_pivot_hcp['total'] = df_pivot_hcp.sum(axis=1)
df_pivot_hcp = df_pivot_hcp.sort_values(['total'], ascending=False)

#Pivot HCO
df_pivot_hco = pd.pivot_table(df_hco, values='value', columns='category', index=['id', 'name', 'address', 'location'])
df_pivot_hco['total'] = df_pivot_hco.sum(axis=1)
df_pivot_hco = df_pivot_hco.sort_values(['total'], ascending=False)

#Write all
writer = pd.ExcelWriter('../../data/99. analyzes/excel/recipients/%s_recipients.xlsx' % year, options={'encoding':'utf-8'})
df_pivot_hcp.to_excel(writer, 'hcp', index=True)
df_pivot_hco.to_excel(writer, 'hco', index=True)
writer.save()

print("finito")

finito
